In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 
plt.style.use('ggplot')
import feature
import warnings
import json
warnings.filterwarnings('ignore')

In [2]:
data = feature.DataCleaner("data.json", "test")

Now after the training and test data have been combined:

The shape of the DataFrame is [2919,79]

The list below are the columns that has NaN values:
MSZoning       4
Utilities      2
Functional     2
KitchenQual    1
SaleType       1
Electrical     1
Exterior1st    1
Exterior2nd    1
dtype: int64


In [ ]:
plt.hist(data.all_data.loc[data.all_data.MiscFeature.isnull(),'MiscVal'], 50, normed = 1, facecolor = 'green')

In [ ]:
max(data.all_data.loc[data.all_data.MiscFeature.isnull(), 'MiscVal'])

In [ ]:
min(data.all_data.loc[data.all_data.MiscFeature.isnull(), 'MiscVal'])

In [ ]:
with open("data.json") as data_file:
    file = json.load(data_file)
data.fill_na_spec(file["fill_dict"])

In [ ]:
group = {("GarageType", "Detchd"): ["GarageArea", "GarageCars"]}
data.fill_na_group(group)

In [ ]:
train_file="../data/train.csv"
test_file="../data/test.csv"
train_data = pd.read_csv(train_file)
test_data = pd.read_csv(test_file)

In [ ]:
train_data['SalePrice']= np.log1p(train_data[train_data.columns.values[-1]])
train_num = train_data.select_dtypes(exclude=["object"]).iloc[:,1:-1]
test_num = test_data.select_dtypes(exclude=["object"]).iloc[:,1:]

simple_numerical(train_num)
simple_numerical(test_num)

train_cat = train_data.select_dtypes(include=["object"])
test_cat = test_data.select_dtypes(include=["object"])

train_cat = simple_categorical(train_cat)
train_cat = simple_categorical(test_cat)

X_train = train_num.join(train_cat)
y_train = train_data['SalePrice']

X_test = test_num.join(test_cat)

In [ ]:
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet,LassoCV,LassoLarsCV
from sklearn.model_selection import cross_val_score

def rmse_cv(model):
    rmse = np.sqrt(-cross_val_score(model, X_train, y_train, scoring = "neg_mean_squared_error", cv = 5))
    return rmse

In [ ]:
model_ridge = Ridge()
alphas = [0.05, 0.1, 0.3, 1, 3, 5, 10, 15, 30, 50, 75]
cv_ridge = [rmse_cv(Ridge(alpha = alpha)).mean() 
            for alpha in alphas]

cv_ridge = pd.Series(cv_ridge, index = alphas)
cv_ridge.plot(title = "Validation - Just Do It")
plt.xlabel("alpha")
plt.ylabel("rmse")